In [8]:
import tarfile
import pandas as pd
import glob
import os
import numpy as np
from collections import defaultdict
import gzip
from helpers_map import *

# MAIN

In [9]:
### Inputs ###

basepath = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374' #TCGA_Ovarian_374' #TCGA_Breast_1102'
filtering_run = 'filtering_samples/filters_19May_order_5ge_wAnnot_GPstar'
fasta_paths = glob.glob(os.path.join(basepath, filtering_run, '*25012024*fa*'))
path_filtered = glob.glob(os.path.join(basepath, filtering_run, '*Filt*'))

In [10]:
### Main ###
for fasta in fasta_paths:
    sample = os.path.basename(fasta).split('_')[1]
    path_filtered_sample = [path for path in path_filtered if sample in path]
    print(len(path_filtered_sample))

    # Fasta
    peptides_IDS, peptides_sequences, peptides_IDS_expand, cuts_peptides = preprocess_fasta(fasta)
    fa = pd.DataFrame({'peptide_id': peptides_IDS_expand, \
    'kmers': cuts_peptides}) 
    fa = fa.drop_duplicates()
    display(fa.head())
    print(fa.shape)
    

    # Experiments
    kmer_to_filesID_ = experiments_preprocess(path_filtered_sample)
    exp = pd.DataFrame(kmer_to_filesID_, columns = ['kmers', 'experiment_ids'])
    display(exp.head())
    print(exp.shape)

    # Result: Merge fasta on experiments
    res = exp.merge(fa, on = 'kmers', how = 'inner')
    display(res.head())
    print(res.shape)
    res = res.drop('kmers', axis = 1).drop_duplicates()
    print(res.shape)
    # Result: Collapse to get unique peptide IDs
    collapse_res = defaultdict(set)
    for pep_id, exp_id in zip(res['peptide_id'], res['experiment_ids']):
        collapse_res[pep_id].update(exp_id.split(';'))
    collapse_res_df = []
    for k, v in collapse_res.items():  # Collapse 
        collapse_res_df.append((k,';'.join(v)))
    collapse_res_df = pd.DataFrame(collapse_res_df, columns = ['peptide_id', 'experiment_ids'])
    display(collapse_res_df.head())
    print(collapse_res_df.shape)
    
    # Result: add peptide sequences
    peptide_sequences_ = pd.DataFrame({'peptide_id': peptides_IDS, 'peptide_sequence':peptides_sequences})
    collapse_res_df = collapse_res_df.merge(peptide_sequences_, on = 'peptide_id', how = 'inner')
    display(collapse_res_df.head())
    print(collapse_res_df.shape)
    
    print('Number of peptide IDS')
    print(len(fa['peptide_id'].unique()))
    print(len(res['peptide_id'].unique()))
    print(len(collapse_res_df['peptide_id'].unique()))
    
    save_name = os.path.join(basepath, filtering_run, f'G_{sample}_experiments_per_peptide_25012024.tsv.gz')
    collapse_res_df.to_csv(save_name, index = None, compression = 'gzip', sep = '\t')
    print(f'Save to {save_name}')
    print('----- \n')
    

#     display(res.loc[res['peptide_id'] == '1'])

65
Load /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-24-1431-01A-01R-1566-13_pool_kmer_25012024.fa.gz fasta


,peptide_id,kmers
0,1,AAPPAQGEP
1,1,AAAPPAQGE
2,1,AAAAPPAQG
3,1,AAAAAPPAQ
4,1,AAAAAAPPA


(1244978, 2)


,kmers,experiment_ids
0,DQPHGPPQQ,02101GA;02102GA;0210AGA;0210XGA;02111GA;02112G...
1,LYPDTATRH,02101GA;02102GA;0210AGA;0210XGA;02111GA;02112G...
2,EVLQGGGPP,02101GA;02102GA;0210AGA;0210XGA;02111GA;02112G...
3,LLAPWQQNS,02101GA;02102GA;0210AGA;0210XGA;02111GA;02112G...
4,PRVQMATDA,02101GA;02102GA;0210AGA;0210XGA;02111GA;02112G...


(58131, 2)


...

(1394534, 3)
(379675, 2)


,peptide_id,experiment_ids
0,249327,0A131GA;0211AGA;0A50AGA;02132GA;0A532GA;0211XG...
1,249328,0A131GA;0211AGA;0A50AGA;02132GA;0A532GA;0211XG...
2,103731,0A131GA;0A50AGA;0A532GA;0A51AGA;0A5AAGA;0AN12G...
3,294014,0A131GA;0211AGA;0A50AGA;02132GA;0A532GA;0211XG...
4,294015,0A131GA;0211AGA;0A50AGA;02132GA;0A532GA;0211XG...


(306906, 2)


,peptide_id,experiment_ids,peptide_sequence
0,249327,0A131GA;0211AGA;0A50AGA;02132GA;0A532GA;0211XG...,MSYGRPPPDVEGMTSLKVDNLTYRTSPDTLRTLWMPWTDAGAA
1,249328,0A131GA;0211AGA;0A50AGA;02132GA;0A532GA;0211XG...,TLWMPWTDAGAA
2,103731,0A131GA;0A50AGA;0A532GA;0A51AGA;0A5AAGA;0AN12G...,GSTYTPSEAGNELDMELGEEEVEEESRPWRCHSPGTAQGHTCLALW...
3,294014,0A131GA;0211AGA;0A50AGA;02132GA;0A532GA;0211XG...,GKPNIGQEQPVDDAAEVPQREPEKERGDPERQREMEEEEDEDEDED...
4,294015,0A131GA;0211AGA;0A50AGA;02132GA;0A532GA;0211XG...,GKPNIGQEQPVDDAAEVPQREPEKERGDPERQREMEEEEDEDEDED...


(331265, 3)
Number of peptide IDS
308377
306906
306906
Save to /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-24-2298-01A-01R-1569-13_experiments_per_peptide_25012024.tsv.gz
----- 



# Debug